In [75]:
import pandas as pd
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
import pprint

boc = BrainObservatoryCache(manifest_file='boc/manifest.json')

# Download a list of all targeted areas
targeted_structures = boc.get_all_targeted_structures()
print("all targeted structures: " + str(targeted_structures)+ "\n")
# Download experiment containers for VISp experiments: (targeted_structures=['VISp'])

# Download a list of all imaging depths
depths = boc.get_all_imaging_depths()
print("all imaging depths: " + str(depths) + "\n")

# Download a list of all cre driver lines 
cre_lines = boc.get_all_cre_lines()
print("all cre lines:\n" + str(cre_lines) )

all targeted structures: ['VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISrl']

all imaging depths: [175, 185, 195, 200, 205, 225, 250, 265, 275, 276, 285, 300, 320, 325, 335, 350, 365, 375, 390, 400, 550, 570, 625]

all cre lines:
['Cux2-CreERT2', 'Emx1-IRES-Cre', 'Fezf2-CreER', 'Nr5a1-Cre', 'Ntsr1-Cre_GN220', 'Pvalb-IRES-Cre', 'Rbp4-Cre_KL100', 'Rorb-IRES2-Cre', 'Scnn1a-Tg3-Cre', 'Slc17a7-IRES2-Cre', 'Sst-IRES-Cre', 'Tlx3-Cre_PL56', 'Vip-IRES-Cre']


## VISp

In [2]:
experiment_containers = boc.get_experiment_containers(targeted_structures = ['VISp'])
specimen_names = [x['donor_name'] for x in experiment_containers]

counts = []
for x in set(specimen_names):
    counts.append(specimen_names.count(x))
max(counts)

3

In [3]:
list(set(specimen_names))[counts.index(3)]
# mouse id for which we have three depths

'373762'

In [4]:
exp_cont_ids = [x['id'] for x in experiment_containers if x['donor_name']=='373762']
exp_cont_ids
# experiment container ids (one for each depth)

[680150731, 676503586, 674679297]

In [5]:
experiment_containers = boc.get_experiment_containers(ids = exp_cont_ids)
experiment_containers_df = pd.DataFrame(experiment_containers, 
                                         columns =['id', 'imaging_depth', 'targeted_structure',
                                                   'cre_line','reporter_line','donor_name',
                                                   'specimen_name','tags','failed'])
experiment_containers_df

,id,imaging_depth,targeted_structure,cre_line,reporter_line,donor_name,specimen_name,tags,failed
0,680150731,375,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),373762,Pvalb-IRES-Cre;Ai162-373762,[],False
1,676503586,275,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),373762,Pvalb-IRES-Cre;Ai162-373762,[],False
2,674679297,200,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),373762,Pvalb-IRES-Cre;Ai162-373762,[],False


In [6]:
experiment_df = pd.DataFrame(columns = ['id', 'imaging_depth', 'targeted_structure',
                                        'cre_line', 'reporter_line', 'acquisition_age_days',
                                        'experiment_container_id', 'session_type', 'donor_name',
                                        'specimen_name', 'fail_eye_tracking'])

traces_df_dict = {"experiment_id" : [], "traces" : [], "stim_table" : []}

In [7]:
for experiment_container_id in exp_cont_ids:
    exp = boc.get_ophys_experiments(experiment_container_ids=[experiment_container_id])

    experiment_df2 = pd.DataFrame(exp,
                                columns = ['id', 'imaging_depth', 'targeted_structure',
                                'cre_line', 'reporter_line', 'acquisition_age_days',
                                'experiment_container_id', 'session_type', 'donor_name',
                                'specimen_name', 'fail_eye_tracking'])
    experiment_df = experiment_df.append(experiment_df2)


In [8]:
experiment_df

,id,imaging_depth,targeted_structure,cre_line,reporter_line,acquisition_age_days,experiment_container_id,session_type,donor_name,specimen_name,fail_eye_tracking
0,681047011,375,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),109,680150731,three_session_C2,373762,Pvalb-IRES-Cre;Ai162-373762,False
1,680150733,375,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),107,680150731,three_session_A,373762,Pvalb-IRES-Cre;Ai162-373762,False
2,680600666,375,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),108,680150731,three_session_B,373762,Pvalb-IRES-Cre;Ai162-373762,False
0,679697901,275,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),106,676503586,three_session_C2,373762,Pvalb-IRES-Cre;Ai162-373762,False
1,679346829,275,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),99,676503586,three_session_B,373762,Pvalb-IRES-Cre;Ai162-373762,False
2,676503588,275,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),98,676503586,three_session_A,373762,Pvalb-IRES-Cre;Ai162-373762,False
0,675062364,200,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),93,674679297,three_session_B,373762,Pvalb-IRES-Cre;Ai162-373762,False
1,676024958,200,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),95,674679297,three_session_C2,373762,Pvalb-IRES-Cre;Ai162-373762,False
2,675477919,200,VISp,Pvalb-IRES-Cre,Ai162(TIT2L-GC6s-ICL-tTA2),94,674679297,three_session_A,373762,Pvalb-IRES-Cre;Ai162-373762,False


In [ ]:
# experiment_containers_df.to_csv('VISp_experiment_containers_df.txt', 
#                                 index=None, sep=' ', mode='a')

In [ ]:
# experiment_df.to_csv('VISp_experiment_df.txt', 
#                       index=None, sep=' ', mode='a')

In [ ]:
# for i in range(len(traces_df_dict["traces"])):
#     trace_df_name = str(traces_df_dict["experiment_id"][i]) + '.txt'
#     traces_df_dict["traces"][i].to_csv(trace_df_name, 
#                                 index=None, sep=' ', mode='a')

### Session A

In [11]:
id_A = experiment_df.loc[experiment_df['session_type'] == 'three_session_A','id']
id_A

1    680150733
2    676503588
2    675477919
Name: id, dtype: object

In [38]:
for i in range(len(id_A)):
    experiment = boc.get_ophys_experiment_data(id_A.iloc[i]) 
    experiment_cells_id = experiment.get_cell_specimen_ids()
    print(len(experiment_cells_id))
    print(len(experiment.get_dff_traces()[0]))

13
115492
24
115481
10
115486


In [39]:
experiment = boc.get_ophys_experiment_data(id_A.iloc[1]) 
experiment_cells_id = experiment.get_cell_specimen_ids()
n_cells = min(15, len(experiment_cells_id))

traces_df = pd.DataFrame(experiment.get_dff_traces()[1][0:n_cells],
                         columns = experiment.get_dff_traces()[0],
                         index = experiment_cells_id[0:n_cells] )
traces_df.shape

(15, 115481)

In [40]:
stim_add = pd.Series(list('n'*len(experiment.get_dff_traces()[0])),
                    index = experiment.get_dff_traces()[0])
stim_list = experiment.list_stimuli()
print(stim_list)

for s in range(len(stim_list)):
    stim_table = experiment.get_stimulus_table(stim_list[s])
    for i in range(stim_table.shape[0]):
        stim_add.iloc[stim_table.at[i,'start']:(stim_table.at[i,'end']+2)] = stim_list[s]

['drifting_gratings', 'natural_movie_one', 'natural_movie_three', 'spontaneous']


In [109]:
stim_dict = {'stimulus':[stim_add.iloc[0]], 
             'start':[0], 
             'end':[],
             'time_start':[stim_add.index[0]], 
             'time_end':[] }
ind = 0
for i in range(len(stim_add)-1):
    if (stim_add.iloc[i+1]!=stim_add.iloc[i]):
        stim_dict['end'].append(i)
        stim_dict['time_end'].append(stim_add.index[i])
        stim_dict['start'].append(i+1)
        stim_dict['time_start'].append(stim_add.index[i+1])
        stim_dict['stimulus'].append(stim_add.iloc[i+1])
        ind = ind+1
stim_dict['end'].append(i+1)
stim_dict['time_end'].append(stim_add.index[i+1]) 

stim_intervals = pd.DataFrame(stim_dict)
stim_intervals['duration'] = stim_intervals['end'] - stim_intervals['start']
stim_intervals

,stimulus,start,end,time_start,time_end,duration
0,n,0,743,14.64826,39.34941,743
1,drifting_gratings,744,805,39.38266,41.41062,61
2,n,806,833,41.44386,42.34148,27
3,drifting_gratings,834,895,42.37473,44.40269,61
4,n,896,923,44.43593,45.33356,27
...,...,...,...,...,...,...
1259,n,115046,115074,3839.49621,3840.42712,28
1260,drifting_gratings,115075,115136,3840.46037,3842.48844,61
1261,n,115137,115164,3842.52168,3843.41935,27
1262,drifting_gratings,115165,115226,3843.45260,3845.48067,61


In [110]:
stim_intervals = stim_intervals.drop(stim_intervals[(stim_intervals.duration<50)&
                                                    (stim_intervals.stimulus=='n')].index)

In [111]:
idx_remove = []

In [112]:
for i in range(stim_intervals.shape[0]-1):
    if(stim_intervals.stimulus.iloc[i] == stim_intervals.stimulus.iloc[i+1]):
        stim_intervals.iat[i,2] = 0
        stim_intervals.iat[i,4] = 0
        stim_intervals.iat[i,5] = 0
        idx_remove.append(stim_intervals.index[i+1])

In [113]:
stim_intervals = stim_intervals.drop(idx_remove, axis=0)

In [114]:
for i in range(stim_intervals.shape[0]-1):
    if(stim_intervals.end.iloc[i] == 0):
        stim_intervals.iat[i,2] = stim_intervals.iat[i+1,1]-1
        stim_intervals.iat[i,4] = stim_intervals.iat[i+1,3]
        stim_intervals.iat[i,5] = stim_intervals.iat[i,2] - stim_intervals.iat[i,1]

In [115]:
stim_intervals

,stimulus,start,end,time_start,time_end,duration
0,n,0,743,14.64826,39.34941,743
1,drifting_gratings,744,18776,39.38266,638.90121,18032
400,n,18777,19708,638.90121,669.85300,931
401,natural_movie_three,19709,37772,669.88624,1270.40574,18063
402,n,37773,38673,1270.43899,1300.36035,900
403,natural_movie_one,38674,47706,1300.39360,1600.67175,9032
404,n,47707,48607,1600.70499,1630.62649,900
405,drifting_gratings,48608,66640,1630.65973,2230.18932,18032
804,n,66641,66788,2230.18932,2235.07654,147
805,spontaneous,66789,75701,2235.10979,2531.40215,8912
